### Importovanje potrebnih lib-ova



In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn.functional as F
from tqdm.auto import tqdm
import numpy as np
import os
try:
    import torchinfo
except:
    !pip install torchinfo
    import torchinfo

from torchinfo import summary

### Setovanje vrednosti za ponovne treninge


In [2]:
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(42)

### Setovanje komponente na kojoj ce se vrsiti ceo proces
GPU ako je dostupan, u suprotnom CPU

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Ucitavanje podataka

In [4]:
BATCH_SIZE = 64
NUM_WORKERS = os.cpu_count()

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

100%|██████████| 169001437/169001437 [00:05<00:00, 29300781.94it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


### Definisanje metoda za treniranje i testiranje

In [5]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
    model.train()

    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        y_pred = model(X)

        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

In [6]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module):
    model.eval()

    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            test_pred_logits = model(X)

            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

In [7]:
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):

    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss, test_acc = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn)

        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results

### Definisanje konstanti


In [8]:
EPOCHS = 200

### Kreiranje glavne klase modela

In [9]:
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class Net(nn.Module):
  def __init__(self, in_channels, num_classes):
    super(Net, self).__init__()
    self.conv1 = conv_block(in_channels, 64)
    self.conv2 = conv_block(64, 128, pool=True)
    self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))

    self.conv3 = conv_block(128, 256, pool=True)
    self.conv4 = conv_block(256, 512, pool=True)
    self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))
    self.conv5 = conv_block(512, 1028, pool=True)
    self.res3 = nn.Sequential(conv_block(1028, 1028), conv_block(1028, 1028))

    self.classifier = nn.Sequential(nn.MaxPool2d(2), # 1028 x 1 x 1
                                    nn.Flatten(), # 1028
                                    nn.Linear(1028, num_classes)) # 1028 -> 100

  def forward(self, xb):
    out = self.conv1(xb)
    out = self.conv2(out)
    out = self.res1(out) + out
    out = self.conv3(out)
    out = self.conv4(out)
    out = self.res2(out) + out
    out = self.conv5(out)
    out = self.res3(out) + out
    out = self.classifier(out)
    return out

### Instanciranje modela

In [10]:
model = Net(3, 100).to(device)

### Pregled modela

In [11]:
summary(model)

Layer (type:depth-idx)                   Param #
Net                                      --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       1,792
│    └─BatchNorm2d: 2-2                  128
│    └─ReLU: 2-3                         --
├─Sequential: 1-2                        --
│    └─Conv2d: 2-4                       73,856
│    └─BatchNorm2d: 2-5                  256
│    └─ReLU: 2-6                         --
│    └─MaxPool2d: 2-7                    --
├─Sequential: 1-3                        --
│    └─Sequential: 2-8                   --
│    │    └─Conv2d: 3-1                  147,584
│    │    └─BatchNorm2d: 3-2             256
│    │    └─ReLU: 3-3                    --
│    └─Sequential: 2-9                   --
│    │    └─Conv2d: 3-4                  147,584
│    │    └─BatchNorm2d: 3-5             256
│    │    └─ReLU: 3-6                    --
├─Sequential: 1-4                        --
│    └─Conv2d: 2-10                      295,168
│

### Treniranje modela i zamrzavanje slojeva

In [14]:
train(model=model,
          train_dataloader=train_loader,
          test_dataloader=test_loader,
          optimizer=torch.optim.Adam(params=model.parameters(), lr=0.0001),
          loss_fn=F.cross_entropy,
          epochs=EPOCHS)

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 3.0112 | train_acc: 0.2699 | test_loss: 2.8367 | test_acc: 0.3150
Epoch: 2 | train_loss: 2.2611 | train_acc: 0.4095 | test_loss: 2.1832 | test_acc: 0.4309
Epoch: 3 | train_loss: 1.8759 | train_acc: 0.4919 | test_loss: 1.9715 | test_acc: 0.4944
Epoch: 4 | train_loss: 1.6191 | train_acc: 0.5488 | test_loss: 1.7331 | test_acc: 0.5365
Epoch: 5 | train_loss: 1.4237 | train_acc: 0.5973 | test_loss: 1.5760 | test_acc: 0.5693
Epoch: 6 | train_loss: 1.2570 | train_acc: 0.6404 | test_loss: 1.4762 | test_acc: 0.5879
Epoch: 7 | train_loss: 1.1123 | train_acc: 0.6759 | test_loss: 1.4364 | test_acc: 0.6065
Epoch: 8 | train_loss: 0.9911 | train_acc: 0.7091 | test_loss: 1.4677 | test_acc: 0.6124
Epoch: 9 | train_loss: 0.8805 | train_acc: 0.7367 | test_loss: 1.5236 | test_acc: 0.6058
Epoch: 10 | train_loss: 0.7668 | train_acc: 0.7669 | test_loss: 1.4095 | test_acc: 0.6408
Epoch: 11 | train_loss: 0.6767 | train_acc: 0.7927 | test_loss: 1.3635 | test_acc: 0.6397
Epoch: 12 | train_l

{'train_loss': [3.0112396899391625,
  2.2611166630559567,
  1.8759187767877603,
  1.6191028690399112,
  1.4237250268001995,
  1.257048896968822,
  1.1122798710070607,
  0.9910725257585725,
  0.8805048835780614,
  0.7668357771604567,
  0.6767172904499351,
  0.5941123332818756,
  0.5106042162193667,
  0.4381349449381804,
  0.3880775520944839,
  0.33530655927251063,
  0.2987882794092988,
  0.2660028118344829,
  0.24465557001531124,
  0.22483866678936706,
  0.19845710200784,
  0.18377035292213226,
  0.1790098868110372,
  0.16472555188428792,
  0.1463459845384597,
  0.14959543441777187,
  0.15158495232415245,
  0.13585439194803653,
  0.12389094273552603,
  0.12239718908215384,
  0.11424110117344581,
  0.11065837781509513,
  0.10983971009731217,
  0.10528351921383339,
  0.10458574932999909,
  0.09530185665244527,
  0.09298088918428135,
  0.09320977576972579,
  0.09474494163890171,
  0.08663890000237413,
  0.08099026675484931,
  0.08729112753108659,
  0.0840677348354979,
  0.07849443629574593

### Cuvanje modela

In [16]:
torch.save(model.state_dict(), 'models/model.pt')

### Ucitavanje modela

In [ ]:
model_normal = Net(3, 100).to(device)
model_normal.load_state_dict(torch.load('models/model.pt'))